In [1]:
import numpy as np
import PIL
import umap
import pandas as pd
import json
import glob
from pytorch_metric_learning import distances, losses, miners, reducers
import torch.nn as nn
import os
import matplotlib.pyplot as plt

In [2]:
import torch
import torchvision
from torchvision import models
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import tqdm
from pytorch_metric_learning import losses, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from pytorch_metric_learning.distances import LpDistance,CosineSimilarity
from pytorch_metric_learning.utils.inference import CustomKNN

In [3]:
%matplotlib inline
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
import numpy as np
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
from torchvision import transforms
import torch.optim as optim
import time
import tqdm as tqdm
from torch.autograd import Variable

In [4]:
import wandb
import random  # for demo script

wandb.login()

wandb: Currently logged in as: pranavjadhav001. Use `wandb login --relogin` to force relogin


True

In [4]:
config = {
    'name':'cub_triplet_loss_epshn_resnet18x512_sgd_aug',
    'dataset':'CUB_200_2011',
    'random_seed':42,
    'model_architecture':'resnet18',
    'embedding_dim':512,
    'distance':'cosine',
    'image_height':224,
    'image_width':224,
    'train_test_split':0.2,
    'class_split':0.1,
    'batch_size':128,
    'optimizer':'sgd',
    'learning_rate':0.001,
    'num_epochs':100,
    'loss':'NTXentLoss',
    'miner':'epshn',
    'reducer':0,
    'metric':'precision_at_1',
    'model_save_path':'models/cub_triplet_loss_epshn_resnet18x512_sgd_aug',
    'temperature': 0.1
}

In [7]:
id = wandb.util.generate_id()
run = wandb.init(
    id=id,
    name = config['name'],
    # Set the project where this run will be logged
    project="embedding_based_classification",
    # Track hyperparameters and run metadata
    config=config,
    resume="allow"
)
print(id)

e14agtu5


In [5]:
np.random.seed(config['random_seed'])
torch.manual_seed(config['random_seed'])
torch.cuda.manual_seed(config['random_seed'])
torch.backends.cudnn.deterministic = False

In [8]:
os.chdir('..')
if not os.path.exists('models'):
    os.makedirs('models')

In [6]:
# print(models.resnet18())
class ResNetFeatrueExtractor18(nn.Module):
    def __init__(self, pretrained = True):
        super(ResNetFeatrueExtractor18, self).__init__()
        model_resnet18 = models.resnet18(pretrained=pretrained)
        self.conv1 = model_resnet18.conv1
        self.bn1 = model_resnet18.bn1
        self.relu = model_resnet18.relu
        self.maxpool = model_resnet18.maxpool
        self.layer1 = model_resnet18.layer1
        self.layer2 = model_resnet18.layer2
        self.layer3 = model_resnet18.layer3
        self.layer4 = model_resnet18.layer4
        self.avgpool = model_resnet18.avgpool
        self.fc1 = nn.Linear(512, config['embedding_dim'])
        
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

In [7]:
ResNetFeatrueExtractor18()(torch.zeros(18,3,28,28)).shape

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([18, 512])

In [8]:
### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
def train(model, loss_func,mining_func, device, train_loader, optimizer, epoch):
    model.train()
    train_losses = []
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(data)
        indices_tuple = mining_func(embeddings, labels)
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()
        train_losses.append(loss)
        if batch_idx % 100 == 0:
            print("Epoch {} Iteration {}: Loss = {}".format(epoch, batch_idx, loss))
    return torch.mean(torch.tensor(train_losses)).item()
    
### convenient function from pytorch-metric-learning ###
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester(dataloader_num_workers=0)
    return tester.get_all_embeddings(dataset, model)

### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, test_labels, train_embeddings, train_labels, False
    )
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))
    return accuracies["precision_at_1"]
    
device = torch.device("cuda")

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# create train and test transforms
transform = transforms.Compose(
    [
        transforms.Resize((config['image_height'], config['image_width'])),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]
)

batch_size = 128

In [18]:
def tra_transforms(imgsize, RGBmean, RGBstdv):
    return transforms.Compose([transforms.Resize(int(imgsize*1.1)),
                                 transforms.RandomCrop(imgsize),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize(RGBmean, RGBstdv)])

def eva_transforms(imgsize, RGBmean, RGBstdv):
    return transforms.Compose([transforms.Resize(imgsize),
                                 transforms.CenterCrop(imgsize),
                                 transforms.ToTensor()])
                                 #transforms.Normalize(RGBmean, RGBstdv)])


In [19]:
train_transform = tra_transforms(224,mean,std)
test_transform = eva_transforms(224,mean,std)

In [20]:
with open('CUB_200_2011/classes.txt','r') as f:
    classes = f.readlines()
classes = [i.replace('\n','') for i in classes]
classes = [i.split(' ')[1] for i in classes]
class_dict = {k:v for k,v in zip(classes,range(200))}

In [21]:
image_paths = []
labels = []
for folder_path,i in class_dict.items():
    folder_images = glob.glob('CUB_200_2011/images/'+'/'+str(folder_path)+'/*')
    image_paths.extend(folder_images)
    labels.extend([i]*len(folder_images))

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_paths, labels, test_size=config['train_test_split'],
                                                    stratify=labels,
                                                    random_state=config['random_seed'])

In [23]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

9430 2358 9430 2358


In [31]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class CUBDataset(Dataset):
    def __init__(self, image_paths,labels,transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        self.load_image_from_paths()
        
    def load_image_from_paths(self):
        self.images = []
        for i in self.image_paths:
            img = PIL.Image.open(i)
            if len(img.getbands()) ==1 :
                img = img.convert("RGB")
            self.images.append(img)
            
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [25]:
train_dataset  = CUBDataset(X_train,y_train,train_transform)
test_dataset  = CUBDataset(X_test,y_test,test_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

In [47]:
model = ResNetFeatrueExtractor18()
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=config['learning_rate'])
num_epochs = config['num_epochs']

### pytorch-metric-learning stuff ###
distance = distances.CosineSimilarity()
reducer = reducers.MeanReducer()
loss_func = losses.NTXentLoss(temperature=config['temperature'], distance=distance, reducer=reducer)
mining_func = miners.BatchEasyHardMiner(pos_strategy="easy",neg_strategy="semihard")
knn_func = CustomKNN(CosineSimilarity())
accuracy_calculator = AccuracyCalculator(include=("precision_at_1",),k=1,knn_func=knn_func,avg_of_avgs=True,return_per_class=False)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
wandb.watch(model, log='all')

[]

In [22]:
#total_loss = []
#total_acc = []
for epoch in range(1, num_epochs + 1):
    train_loss = train(model, loss_func,mining_func, device, train_loader, optimizer, epoch)
    #total_loss.extend(train_loss)
    test_acc = test(train_dataset, test_dataset, model, accuracy_calculator)
    #total_acc.append(test_acc)
    wandb.log({"test_accuracy": test_acc, "train_loss": train_loss,'epoch':epoch})

Epoch 1 Iteration 0: Loss = 0.6506922841072083


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:09<00:00,  7.74it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.4202739898989899
Epoch 2 Iteration 0: Loss = 0.6294650435447693


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:03<00:00, 18.56it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.425145202020202
Epoch 3 Iteration 0: Loss = 0.6085958480834961


...

Computing accuracy
Test set accuracy (Precision@1) = 0.6693661616161617
Epoch 99 Iteration 0: Loss = 0.41386568546295166


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:03<00:00, 19.32it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.6629040404040404
Epoch 100 Iteration 0: Loss = 0.38302481174468994


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:03<00:00, 19.56it/s]

Computing accuracy
Test set accuracy (Precision@1) = 0.6593396464646466


In [24]:
wandb.unwatch()

In [25]:
torch.save(model,config['model_save_path']+'_200.pth')

In [19]:
model = torch.load('models/cub_triplet_loss_epshn_resnet18_sgd_aug_200.pth',map_location='cuda')

In [26]:
model.eval()

ResNetFeatrueExtractor18(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReL

In [27]:
from pytorch_metric_learning.distances import LpDistance,CosineSimilarity
from pytorch_metric_learning.utils.inference import CustomKNN
knn_func = CustomKNN(CosineSimilarity())
accuracy_calculator = AccuracyCalculator(include=("precision_at_1",),k=1,knn_func=knn_func,avg_of_avgs=False,return_per_class=True)
#test(train_dataset, test_dataset, model, accuracy_calculator)

In [28]:
train_embeddings, train_labels = get_all_embeddings(train_dataset, model)
test_embeddings, test_labels = get_all_embeddings(test_dataset, model)
train_labels = train_labels.squeeze(1)
test_labels = test_labels.squeeze(1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:03<00:00, 19.05it/s]


In [29]:
accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, test_labels, train_embeddings, train_labels)
print(accuracies)

{'precision_at_1': [0.75, 0.75, 0.6666666666666666, 0.8333333333333334, 0.6666666666666666, 0.75, 1.0, 0.5555555555555556, 0.4166666666666667, 0.8333333333333334, 0.5, 1.0, 0.75, 0.75, 0.7272727272727273, 0.7272727272727273, 0.6363636363636364, 1.0, 0.75, 0.5, 0.75, 0.5454545454545454, 0.4166666666666667, 0.8, 0.4166666666666667, 0.5833333333333334, 0.25, 1.0, 0.25, 0.16666666666666666, 0.6666666666666666, 0.6, 0.75, 0.8333333333333334, 0.9166666666666666, 0.6666666666666666, 0.3333333333333333, 0.5833333333333334, 0.4166666666666667, 0.25, 0.9166666666666666, 1.0, 0.16666666666666666, 0.8333333333333334, 0.5, 0.9166666666666666, 0.75, 1.0, 0.25, 0.75, 0.5, 0.8333333333333334, 0.8333333333333334, 0.5833333333333334, 1.0, 0.75, 0.9166666666666666, 0.75, 0.4166666666666667, 0.16666666666666666, 0.8333333333333334, 0.16666666666666666, 1.0, 0.4166666666666667, 0.4, 0.4166666666666667, 0.4166666666666667, 0.6666666666666666, 0.8333333333333334, 0.9166666666666666, 0.25, 0.4166666666666667,

In [30]:
data_dict = {v:k for k,v in class_dict.items()}

In [31]:
acc_dict = {}
for i in test_labels.unique():
    new_labels = test_labels[test_labels==i]
    new_embeddings = test_embeddings[test_labels==i]
    accuracies = accuracy_calculator.get_accuracy(
        new_embeddings, new_labels, train_embeddings, train_labels, False
    )
    acc_dict[data_dict[int(i.detach().cpu().numpy())]]=[ 
                  len(new_labels),
                  len(train_labels[train_labels==i]),
                  accuracies["precision_at_1"][0]]
    print("{:<30} test samples {:<5}, training samples {:<5}: {}".format(data_dict[int(i.detach().cpu().numpy())],
                                                                  len(new_labels),
                                                                  len(train_labels[train_labels==i]),
                                                                  accuracies["precision_at_1"]))

001.Black_footed_Albatross     test samples 12   , training samples 48   : [0.75]
002.Laysan_Albatross           test samples 12   , training samples 48   : [0.75]
003.Sooty_Albatross            test samples 12   , training samples 46   : [0.6666666666666666]
004.Groove_billed_Ani          test samples 12   , training samples 48   : [0.8333333333333334]
005.Crested_Auklet             test samples 9    , training samples 35   : [0.6666666666666666]
006.Least_Auklet               test samples 8    , training samples 33   : [0.75]
007.Parakeet_Auklet            test samples 10   , training samples 43   : [1.0]
008.Rhinoceros_Auklet          test samples 9    , training samples 39   : [0.5555555555555556]
009.Brewer_Blackbird           test samples 12   , training samples 47   : [0.4166666666666667]
010.Red_winged_Blackbird       test samples 12   , training samples 48   : [0.8333333333333334]
011.Rusty_Blackbird            test samples 12   , training samples 48   : [0.5]
012.Yellow_heade

In [32]:
all_table = [[k]+v for k,v in acc_dict.items()]

In [33]:
columns = ["class_name", "no. of test samples", "no. of train samples", "precision@1"]
train_table = wandb.Table(data=all_table, columns=columns)

In [34]:
run.log({"all_classes_metrics": train_table})

In [35]:
train_embeddings, train_labels = get_all_embeddings(train_dataset, model)
test_embeddings, test_labels = get_all_embeddings(test_dataset, model)
train_labels = train_labels.squeeze(1)
test_labels = test_labels.squeeze(1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:03<00:00, 18.84it/s]


In [36]:
import faiss
# Create a Faiss index
index = faiss.IndexFlatIP(config['embedding_dim'])
# Add some vectors to the index
index.add(train_embeddings.detach().cpu().numpy())

In [37]:
pred_labels = [] 
for embedding,label in zip(test_embeddings.detach().cpu().numpy(),test_labels):
    _, indices = index.search(embedding.reshape(1,-1).astype(np.float32), 1)
    pred_class = train_labels[indices[0][0]]
    pred_labels.append(pred_class)

In [38]:
pred_labels = [i.detach().cpu().numpy() for i in pred_labels]

In [39]:
from sklearn.metrics import classification_report
report = classification_report(test_labels.detach().cpu().numpy(), pred_labels, target_names=list(class_dict.keys()),output_dict=True)

In [40]:
print(classification_report(test_labels.detach().cpu().numpy(), pred_labels, target_names=list(class_dict.keys())))

                                    precision    recall  f1-score   support

        001.Black_footed_Albatross       0.59      0.83      0.69        12
              002.Laysan_Albatross       0.53      0.75      0.62        12
               003.Sooty_Albatross       0.90      0.75      0.82        12
             004.Groove_billed_Ani       0.59      0.83      0.69        12
                005.Crested_Auklet       1.00      0.67      0.80         9
                  006.Least_Auklet       0.86      0.75      0.80         8
               007.Parakeet_Auklet       0.77      1.00      0.87        10
             008.Rhinoceros_Auklet       0.55      0.67      0.60         9
              009.Brewer_Blackbird       0.47      0.58      0.52        12
          010.Red_winged_Blackbird       0.82      0.75      0.78        12
               011.Rusty_Blackbird       0.44      0.33      0.38        12
       012.Yellow_headed_Blackbird       1.00      1.00      1.00        11
           

In [41]:
df = pd.DataFrame(report).transpose()

In [42]:
df.head()

,precision,recall,f1-score,support
001.Black_footed_Albatross,0.588235,0.833333,0.689655,12.0
002.Laysan_Albatross,0.529412,0.750000,0.620690,12.0
003.Sooty_Albatross,0.900000,0.750000,0.818182,12.0
004.Groove_billed_Ani,0.588235,0.833333,0.689655,12.0
005.Crested_Auklet,1.000000,0.666667,0.800000,9.0


In [43]:
df.reset_index(inplace=True)

In [44]:
df.rename(columns={"index":'class_name'},inplace=True)

In [45]:
classification_report_table = wandb.Table(dataframe=df)

In [46]:
run.log({"all_classes_classification_report": classification_report_table})

## Removing classes from training dataset and seeing performance

In [47]:
train_image_paths = []
train_labels = []
for folder_path,i in class_dict.items():
    if i >= 180:
        break
    folder_images = glob.glob('CUB_200_2011/images/'+'/'+str(folder_path)+'/*')
    train_image_paths.extend(folder_images)
    train_labels.extend([i]*len(folder_images))

In [48]:
test_image_paths = []
test_labels = []
for folder_path,i in class_dict.items():
    if i >= 180:
        folder_images = glob.glob('CUB_200_2011/images/'+'/'+str(folder_path)+'/*')
        test_image_paths.extend(folder_images)
        test_labels.extend([i]*len(folder_images))

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_image_paths,train_labels, test_size=config['train_test_split'],
                                                    stratify=train_labels, random_state=config['random_seed'])

In [50]:
train_dataset  = CUBDataset(X_train,y_train,train_transform)
test_dataset  = CUBDataset(X_test,y_test,test_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)

In [51]:
model = ResNetFeatrueExtractor18()
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=config['learning_rate'])
num_epochs = config['num_epochs']

### pytorch-metric-learning stuff ###
distance = distances.CosineSimilarity()
reducer = reducers.MeanReducer()
loss_func = losses.NTXentLoss(temperature=config['temperature'], distance=distance, reducer=reducer)
mining_func = miners.BatchEasyHardMiner(pos_strategy="easy",neg_strategy="semihard")
knn_func = CustomKNN(CosineSimilarity())
accuracy_calculator = AccuracyCalculator(include=("precision_at_1",),k=1,knn_func=knn_func,avg_of_avgs=True,return_per_class=False)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [52]:
#total_loss = []
#total_acc = []
for epoch in range(1, num_epochs + 1):
    train_loss = train(model, loss_func,mining_func, device, train_loader, optimizer, epoch)
    #total_loss.extend(train_loss)
    test_acc = test(train_dataset, test_dataset, model, accuracy_calculator)
    #total_acc.append(test_acc)
    wandb.log({"test_accuracy2": test_acc, "train_loss2": train_loss,"epoch": epoch})

Epoch 1 Iteration 0: Loss = 0.6358517408370972


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:08<00:00,  7.55it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.4199228395061728
Epoch 2 Iteration 0: Loss = 0.6389706134796143


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:03<00:00, 19.06it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.4416526374859708
Epoch 3 Iteration 0: Loss = 0.6190959811210632


...

Computing accuracy
Test set accuracy (Precision@1) = 0.6661826599326599
Epoch 99 Iteration 0: Loss = 0.4709480404853821


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:03<00:00, 17.78it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.6596618967452301
Epoch 100 Iteration 0: Loss = 0.44111236929893494


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:03<00:00, 17.92it/s]

Computing accuracy
Test set accuracy (Precision@1) = 0.6748302469135802


In [53]:
torch.save(model,config['model_save_path']+'_180.pth')

In [32]:
model = torch.load('models/cub_triplet_loss_epshn_resnet18_sgd_aug_180.pth',map_location='cuda')

In [54]:
model.eval()

ResNetFeatrueExtractor18(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReL

In [55]:
image_paths = []
labels = []
for folder_path,i in class_dict.items():
    folder_images = glob.glob('CUB_200_2011/images/'+'/'+str(folder_path)+'/*')
    image_paths.extend(folder_images)
    labels.extend([i]*len(folder_images))

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_paths, labels, test_size=config['train_test_split'],
                                                    stratify=labels,
                                                    random_state=config['random_seed'])

In [57]:
train_dataset  = CUBDataset(X_train,y_train,transform)
test_dataset  = CUBDataset(X_test,y_test,transform)

In [58]:
train_embeddings, train_labels = get_all_embeddings(train_dataset, model)
test_embeddings, test_labels = get_all_embeddings(test_dataset, model)
train_labels = train_labels.squeeze(1)
test_labels = test_labels.squeeze(1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:09<00:00,  7.71it/s]


In [59]:
acc_dict2 = {}
for i in test_labels.unique():
    new_labels = test_labels[test_labels==i]
    new_embeddings = test_embeddings[test_labels==i]
    accuracies = accuracy_calculator.get_accuracy(
        new_embeddings, new_labels, train_embeddings, train_labels, False
    )
    acc_dict2[data_dict[int(i.detach().cpu().numpy())]]=[ 
                  len(new_labels),
                  len(train_labels[train_labels==i]),
                  accuracies["precision_at_1"]]
    print("{:<30} test samples {:<5}, training samples {:<5}: {}".format(data_dict[int(i.detach().cpu().numpy())],
                                                                  len(new_labels),
                                                                  len(train_labels[train_labels==i]),
                                                                  accuracies["precision_at_1"]))

001.Black_footed_Albatross     test samples 12   , training samples 48   : 0.75
002.Laysan_Albatross           test samples 12   , training samples 48   : 0.75
003.Sooty_Albatross            test samples 12   , training samples 46   : 0.5833333333333334
004.Groove_billed_Ani          test samples 12   , training samples 48   : 0.8333333333333334
005.Crested_Auklet             test samples 9    , training samples 35   : 0.8888888888888888
006.Least_Auklet               test samples 8    , training samples 33   : 0.875
007.Parakeet_Auklet            test samples 10   , training samples 43   : 1.0
008.Rhinoceros_Auklet          test samples 9    , training samples 39   : 0.3333333333333333
009.Brewer_Blackbird           test samples 12   , training samples 47   : 0.4166666666666667
010.Red_winged_Blackbird       test samples 12   , training samples 48   : 0.8333333333333334
011.Rusty_Blackbird            test samples 12   , training samples 48   : 0.4166666666666667
012.Yellow_headed_Blac

In [60]:
all_table = [[k]+v for k,v in acc_dict2.items()]
columns = ["class_name", "no. of test samples", "no. of train samples", "precision@1"]
train_table = wandb.Table(data=all_table, columns=columns)
run.log({"limited_classes_metrics": train_table})

In [61]:
import faiss
# Create a Faiss index
index = faiss.IndexFlatIP(config['embedding_dim'])
# Add some vectors to the index
index.add(train_embeddings.detach().cpu().numpy())

In [62]:
pred_labels = [] 
for embedding,label in zip(test_embeddings.detach().cpu().numpy(),test_labels):
    distances, indices = index.search(embedding.reshape(1,-1).astype(np.float32), 2)
    pred_class = train_labels[indices[0][1]]
    pred_labels.append(pred_class)

In [63]:
pred_labels = [i.detach().cpu().numpy() for i in pred_labels]

In [64]:
from sklearn.metrics import classification_report
print(classification_report(test_labels.detach().cpu().numpy(), pred_labels, target_names=list(class_dict.keys())))

                                    precision    recall  f1-score   support

        001.Black_footed_Albatross       0.38      0.75      0.50        12
              002.Laysan_Albatross       0.43      0.50      0.46        12
               003.Sooty_Albatross       0.36      0.33      0.35        12
             004.Groove_billed_Ani       0.48      0.83      0.61        12
                005.Crested_Auklet       0.75      0.67      0.71         9
                  006.Least_Auklet       0.60      0.75      0.67         8
               007.Parakeet_Auklet       0.80      0.80      0.80        10
             008.Rhinoceros_Auklet       0.50      0.22      0.31         9
              009.Brewer_Blackbird       0.40      0.33      0.36        12
          010.Red_winged_Blackbird       0.90      0.75      0.82        12
               011.Rusty_Blackbird       0.57      0.33      0.42        12
       012.Yellow_headed_Blackbird       0.83      0.91      0.87        11
           

In [65]:
from sklearn.metrics import classification_report
report = classification_report(test_labels.detach().cpu().numpy(), pred_labels, target_names=list(class_dict.keys()),output_dict=True)
df = pd.DataFrame(report).transpose()
df.reset_index(inplace=True)
df.rename(columns={"index":'class_name'},inplace=True)
classification_report_table = wandb.Table(dataframe=df)

In [66]:
run.log({"limited_classes_classification_report": classification_report_table})

## metric drop for unseen classes

In [83]:
unseen_class_names = sorted(key for key in acc_dict.keys() if int(key.split('.')[0]) >= 180)

In [84]:
comparison_table_data = []
for class_name in unseen_class_names:
    comparison_table_data.append([class_name,acc_dict[class_name][-1],acc_dict2[class_name][-1]])

In [85]:
comparison_table_data

[['180.Wilson_Warbler', 0.3333333333333333, 0.75],
 ['181.Worm_eating_Warbler', 0.6666666666666666, 0.5833333333333334],
 ['182.Yellow_Warbler', 0.5, 0.5833333333333334],
 ['183.Northern_Waterthrush', 0.6666666666666666, 0.75],
 ['184.Louisiana_Waterthrush', 0.4166666666666667, 0.25],
 ['185.Bohemian_Waxwing', 0.9166666666666666, 0.75],
 ['186.Cedar_Waxwing', 0.8333333333333334, 0.75],
 ['187.American_Three_toed_Woodpecker', 0.9, 0.7],
 ['188.Pileated_Woodpecker', 0.8333333333333334, 0.6666666666666666],
 ['189.Red_bellied_Woodpecker', 1.0, 1.0],
 ['190.Red_cockaded_Woodpecker', 0.9090909090909091, 0.7272727272727273],
 ['191.Red_headed_Woodpecker', 0.75, 0.75],
 ['192.Downy_Woodpecker', 1.0, 1.0],
 ['193.Bewick_Wren', 0.5, 0.4166666666666667],
 ['194.Cactus_Wren', 0.9166666666666666, 0.5],
 ['195.Carolina_Wren', 0.9166666666666666, 0.6666666666666666],
 ['196.House_Wren', 0.6666666666666666, 0.25],
 ['197.Marsh_Wren', 0.5833333333333334, 0.4166666666666667],
 ['198.Rock_Wren', 0.66666

In [71]:
comparison_table = wandb.Table(data=comparison_table_data, columns=['class_name','all_classes_precision@1','unseen_classes_precision@1'])

In [72]:
run.log({"comparison_unseen_classes_metrics": comparison_table})

In [73]:
precisions = np.array(comparison_table_data)[:,1:3].astype(np.float32)

In [74]:
wandb.log({'precision_drop_unseen_classes':np.mean(np.subtract(precisions[:,0],precisions[:,1]))})

In [87]:
np.mean(np.array(comparison_table_data)[:,1].astype(np.float32))

0.74884564

In [88]:
table = wandb.Table(data=[['trained',round(np.mean(np.array(comparison_table_data)[:,1].astype(np.float32)),2)],
                          ['untrained',np.mean(np.array(comparison_table_data)[:,2].astype(np.float32))]],
                    columns = ["label", "value"])

In [89]:
wandb.log({"avg_precision_unseen_classes_chart" : wandb.plot.bar(table, "label", "value",title="avg_precision_unseen_classes_chart")})

## metric drop for all classes because of new unseen classes

In [76]:
comparison_table_data = []
for class_name in acc_dict.keys():
    comparison_table_data.append([class_name,acc_dict[class_name][-1],acc_dict2[class_name][-1]])

In [77]:
precisions = np.array(comparison_table_data)[:,1:3].astype(np.float32)

In [78]:
precisions

array([[0.75      , 0.75      ],
       [0.75      , 0.75      ],
       [0.6666667 , 0.5833333 ],
       [0.8333333 , 0.8333333 ],
       [0.6666667 , 0.8888889 ],
       [0.75      , 0.875     ],
       [1.        , 1.        ],
       [0.5555556 , 0.33333334],
       [0.41666666, 0.41666666],
       [0.8333333 , 0.8333333 ],
       [0.5       , 0.41666666],
       [1.        , 0.90909094],
       [0.75      , 0.6666667 ],
       [0.75      , 0.9166667 ],
       [0.72727275, 0.72727275],
       [0.72727275, 0.6363636 ],
       [0.6363636 , 0.45454547],
       [1.        , 0.8888889 ],
       [0.75      , 0.6666667 ],
       [0.5       , 0.5       ],
       [0.75      , 0.6666667 ],
       [0.54545456, 0.36363637],
       [0.41666666, 0.33333334],
       [0.8       , 0.8       ],
       [0.41666666, 0.08333334],
       [0.5833333 , 0.5833333 ],
       [0.25      , 0.16666667],
       [1.        , 1.        ],
       [0.25      , 0.33333334],
       [0.16666667, 0.        ],
       [0.

In [79]:
wandb.log({'precision_drop_all_classes':np.mean(np.subtract(precisions[:,0],precisions[:,1]))})

In [90]:
run.finish()

epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇█▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇██
precision_drop_all_classes,▁
precision_drop_unseen_classes,▁
test_accuracy,▁▁▂▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████
test_accuracy2,▁▂▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▆▇▇▇▇▇▇▇█▇██████
train_loss,██▇▇▇▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
train_loss2,██▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁
epoch,100
precision_drop_all_classes,0.04457
precision_drop_unseen_classes,0.10548
test_accuracy,0.65934
